In [1]:
using Pkg; Pkg.activate(".")

  Activating project at `~/code/julia/notebooks`


In [2]:
using OperationsResearchModels

|  Activity   |  Optimistic  |  Most Likely |  Pessimistic | Dependencies  |
| :---------: | :---------:  | :----------: | :----------: | :----------:  |
|      A      |    1         |     3        |    4         |    -          |
|      B      |    2         |     3        |    3         |    -          |
|      C      |    4         |     5        |    5         |   A, B        |
|      D      |    3         |     3        |    5         |    B          |
|      E      |    2         |     2        |    3         |   C, D        |
|      F      |    4         |     4        |    4         |   B, C        |
|      G      |    2         |     4        |    6         |   A, D        |
|      H      |    1         |     5        |    6         |   B, C, D     |
|      I      |    2         |     4        |    5         |   F, G, H     |
|      J      |    5         |     5        |    6         |   A, B, D, H  |
|      K      |    1         |     2        |    4         |   A, G, I     |

In [3]:
begin
    A = PertActivity("A", 1, 3, 4, [])
    B = PertActivity("B", 2, 3, 3, [])
    C = PertActivity("C", 4, 5, 5, [A, B])
    D = PertActivity("D", 3, 3, 5, [B])
    E = PertActivity("E", 2, 2, 3, [C, D])
    F = PertActivity("F", 4, 4, 4, [B, C])
    G = PertActivity("G", 2, 4, 6, [A, D])
    H = PertActivity("H", 1, 5, 6, [B, C, D])
    I = PertActivity("I", 2, 4, 5, [F, G, H])
    J = PertActivity("J", 5, 5, 6, [A, B, D, H])
    K = PertActivity("K", 1, 2, 4, [A, G, I])
end

PertActivity("K", 1.0, 2.0, 4.0, PertActivity[PertActivity("A", 1.0, 3.0, 4.0, PertActivity[]), PertActivity("G", 2.0, 4.0, 6.0, PertActivity[PertActivity("A", 1.0, 3.0, 4.0, PertActivity[]), PertActivity("D", 3.0, 3.0, 5.0, PertActivity[PertActivity("B", 2.0, 3.0, 3.0, PertActivity[])])]), PertActivity("I", 2.0, 4.0, 5.0, PertActivity[PertActivity("F", 4.0, 4.0, 4.0, PertActivity[PertActivity("B", 2.0, 3.0, 3.0, PertActivity[]), PertActivity("C", 4.0, 5.0, 5.0, PertActivity[PertActivity("A", 1.0, 3.0, 4.0, PertActivity[]), PertActivity("B", 2.0, 3.0, 3.0, PertActivity[])])]), PertActivity("G", 2.0, 4.0, 6.0, PertActivity[PertActivity("A", 1.0, 3.0, 4.0, PertActivity[]), PertActivity("D", 3.0, 3.0, 5.0, PertActivity[PertActivity("B", 2.0, 3.0, 3.0, PertActivity[])])]), PertActivity("H", 1.0, 5.0, 6.0, PertActivity[PertActivity("B", 2.0, 3.0, 3.0, PertActivity[]), PertActivity("C", 4.0, 5.0, 5.0, PertActivity[PertActivity("A", 1.0, 3.0, 4.0, PertActivity[]), PertActivity("B", 2.0, 3.0, 

In [4]:
problem = PertProblem([A, B, C, D, E, F, G, H, I, J, K]);

In [5]:
result = solve(problem);

In [6]:
map(x->x.name, result.path)

5-element Vector{String}:
 "A"
 "C"
 "H"
 "I"
 "K"

In [7]:
result.mean

18.166666666666668

In [8]:
result.stddev

1.2133516482134197

In [9]:
pertmean(x::PertActivity) = (x.optimistic + 4 * x.mostlikely + x.pessimistic)/6    

pertmean (generic function with 1 method)

In [10]:
pertvariance(x::PertActivity) = ((x.pessimistic - x.optimistic) / 6)^2

pertvariance (generic function with 1 method)

In [11]:
pertmean(A) + pertmean(C) + pertmean(H) + pertmean(I) + pertmean(K)

18.166666666666668

In [12]:
pertstdev = sqrt(pertvariance(A) + pertvariance(C) + pertvariance(H) + pertvariance(I) + pertvariance(K))

1.2133516482134197

In [13]:
# 95% confidence interval of average earliest completion time

In [14]:
(result.mean - 1.96 * result.stddev, result.mean + 1.96 * result.stddev)

(15.788497436168365, 20.54483589716497)

In [15]:
?solve(::PertProblem)

```julia
solve(problem::PertProblem)::PertResult
```

# Arguments

  * `problem::PertProblem`: The problem in type of PertProblem.

# Output

  * `::PertResult`: The object holds the results

# Example

```julia
julia> A = PertActivity("A", 1, 2, 3)
PertActivity("A", 1.0, 2.0, 3.0, PertActivity[])

julia> B = PertActivity("B", 3, 3, 3)
PertActivity("B", 3.0, 3.0, 3.0, PertActivity[])

julia> C = PertActivity("C", 5, 5, 5, [A, B])
PertActivity("C", 5.0, 5.0, 5.0, PertActivity[PertActivity("A", 1.0, 2.0, 3.0, PertActivity[]), PertActivity("B", 3.0, 3.0, 3.0, PertActivity[])])

julia> activities = [A, B, C]
3-element Vector{PertActivity}:
 PertActivity("A", 1.0, 2.0, 3.0, PertActivity[])
 PertActivity("B", 3.0, 3.0, 3.0, PertActivity[])
 PertActivity("C", 5.0, 5.0, 5.0, PertActivity[PertActivity("A", 1.0, 2.0, 3.0, PertActivity[]), PertActivity("B", 3.0, 3.0, 3.0, PertActivity[])])

julia> problem = PertProblem(activities);

julia> result = pert(activities)
PertResult(PertActivity[PertActivity("B", 3.0, 3.0, 3.0, PertActivity[]), PertActivity("C", 5.0, 5.0, 5.0, PertActivity[PertActivity("A", 1.0, 2.0, 3.0, PertActivity[]), PertActivity("B", 3.0, 3.0, 3.0, PertActivity[])])], 8.0, 0.0)

julia> result.mean
8.0

julia> result.stddev
0.0
```
